### CEBD 1260 Final Programming Assignment

### Part 1 (K-Nearest Neighbors) ###

1) First task to is to classify data from a cancer diagnostic database. In this database are patients with tumors, characteristics of those tumors, and biospy results indicating whether the tumor is Malignant or Benign.


__** About the dataset **__


In cancer_data.txt you will find the following variables:

   - radius (mean of distances from center to points on the perimeter)
   - texture (standard deviation of gray-scale values)
   - perimeter
   - area
   - smoothness (local variation in radius lengths)
   - compactness (perimeter^2 / area - 1.0)
   - concavity (severity of concave portions of the contour)
   - concave_points (number of concave portions of the contour)
   - symmetry 
   - fractal_dimension ("coastline approximation" - 1)
   - cancer (0 = Benign, 1 = Malignant)  *target*


Use any machine learning algorithm you wish. In your answer include a short description of your algorithm of choice and predicted category of a new patient with a tumor with the following features:

   - radius: 14
   - texture: 14
   - perimeter: 88
   - area: 566
   - smoothness: 1
   - compactness: 0.08
   - concavity: 0.06
   - concae points: 0.04
   - symmetry: 0.18
   - fractal dimension: 0.05

In [10]:
#Import Spark libraries/packages

from pyspark import SparkContext
sc = SparkContext()

In [11]:
import pandas as pd

#Read training data
df = pd.read_csv("Data/cancer_data.csv")
      
#Get dataset information   
print(len(df))
print(df.dtypes)
df.head()

569
radius               float64
texture              float64
perimeter            float64
area                 float64
smoothness           float64
compactness          float64
concavity            float64
concave_points       float64
symmetry             float64
fractal_dimension    float64
cancer               float64
dtype: object


,radius,texture,perimeter,area,smoothness,compactness,concavity,concave_points,symmetry,fractal_dimension,cancer
0,18.0,10.4,123.0,1000.0,0.1180,0.2780,0.3000,0.1470,0.242,0.0787,0.0
1,20.6,17.8,133.0,1330.0,0.0847,0.0786,0.0869,0.0702,0.181,0.0567,0.0
2,19.7,21.3,130.0,1200.0,0.1100,0.1600,0.1970,0.1280,0.207,0.0600,0.0
3,11.4,20.4,77.6,386.0,0.1420,0.2840,0.2410,0.1050,0.260,0.0974,0.0
4,20.3,14.3,135.0,1300.0,0.1000,0.1330,0.1980,0.1040,0.181,0.0588,0.0


In [12]:
import numpy as np
#from sklearn.cross_validation import train_test_split (deprecated)
from sklearn.model_selection import train_test_split

#set k initially
k = 3

# Split the data into train and test for:
# 1. A training dataset that kNN to use to make predictions 
# 2. A test dataset that we can use to evaluate the accuracy of the model

X_train = df.iloc[:,:10]
ys = df['cancer']
y_train= ys.to_frame(name='cancer')

# create design matrix X and target vector y 
X = np.array(df.iloc[:,:10].values) 
y = np.array(df['cancer'])  #separate our data fom actual targets themselves

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print(type(X))
print(type(y))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [17]:
# loading library
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# instantiate learning model (k = 3)
knn = KNeighborsClassifier(n_neighbors=3, weights='distance')

# fitting the model
knn.fit(X_train, y_train)

# predict the response
pred = knn.predict(X_test)

# evaluate accuracy
print('The accuracy classification core is: ', accuracy_score(y_test, pred))

The accuracy classification core is:  0.898936170213


In [18]:
#Predict category of a new patient with a tumor with the features listed.

#Predict the class label (cancer (0 = Benign, 1 = Malignant) target) to which unseen samples belong

# radius: 14  texture: 14 perimeter: 88 area: 566 smoothness: 1 compactness: 0.08
# concavity: 0.06 concae points: 0.04 symmetry: 0.18 fractal dimension: 0.05

feature = [14,14,88,566,1,0.08,0.06,0.04,0.18,0.05]

pred = knn.predict([feature])
print('The category of the new patient is:', pred[0])
#type(pred)

The category of the new patient is: 1.0


##### Predicted summary #####
The class label for the sample data provided is 1 (class 1 = Malignant).

In [19]:
sc.stop()

### Part 2 (Support Vector Machine) ###

The following code contains a 5 bugs (errors). Find and correct them all and then answer the following questions:

  1. How many observations are in the training dataset?
  2. How many features are in the training dataset?
  3. How well did your model perform?

  BONUS: Which category is Hockey? 0 or 1? Which category is baseball?

#### List of bug corrections

(1) Modified statement 
from: sklearn.pipeline import pipeline 
to: from sklearn.pipeline import Pipeline

(2) Modified statement
from: sklearn_linear_model import SGDClassifier 
to: from sklearn.linear_model import SGDClassifier

(3) Changed dataset name to reflect subset data extracted: 
    twenty_train = fetch_20newsgroups(subset='test', categories=categories) to:
    twenty_test = fetch_20newsgroups(subset='test', categories=categories)

(4) Changed dataset name to subset subset data extracted:
    twenty_test = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42) to:
    twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

(5) Modified statement:
    text_clf.fit(twenty_train.data, twenty_test.target) to :
    text_clf.fit(twenty_train.data, twenty_train.target) 
    
    reason=> the parameters to the fit method are the training data and the target values of its train samples.


In [20]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier

categories = [ 'rec.sport.baseball','rec.sport.hockey']

twenty_test = fetch_20newsgroups(subset='test', categories=categories)
twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)

#Some descriptions of the sklearn bunch dataset

type(twenty_train)
len(twenty_train) # 6 columns/attributes
print(twenty_train.keys())


dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR', 'description'])


#### 1. How many observations are in the training dataset?

In [21]:
#1 How many observations are in the training dataset?

l=len(twenty_train['data'])
print("The number of observations are", l)

#for key, value in twenty_train.items() :
#    print (key, value)


The number of observations are 1197


#### 2. How many features are in the training dataset?

In [22]:
#2 How many features are in the training dataset?

# twenty_train.data contains the features (data of raw text used for classification)
# print(twenty_train.data)

#The data "twenty_train" is a 2 D array with the shape (number of samples, number of features). 
#In our case, a sample of filenames of shape (1197,) - the files are loaded in data attribute

l1 = len(twenty_train.data)
l2 = len(twenty_train.filenames)
num_features=twenty_train.filenames.shape

print("The number of features is", num_features[0]) #tuple: (1197,)


#Some checking statements
#print("l1=",l1)
#print("l2=",l1)
#print(twenty_train.target[0], twenty_train.data[0])
#print(twenty_train.filenames[0])
#type(twenty_train.filenames) #numpy.ndarray


The number of features is 1197


#### 3. How well did your model perform?

In [23]:
#Convert a collection of text documents to a matrix of token counts
#frequency count (and not tf-idf)

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape 
type(X_train_counts)

#dimension 1197 arcticles, 18571 words/stems

print("Dimension is " ,X_train_counts.shape[0], " arcticles,", X_train_counts.shape[1], "words/stems")


Dimension is  1197  arcticles, 18571 words/stems


In [24]:
# Use support vector machines algorithm to evaluate the performance the text classification algorithm model

import numpy as np
from sklearn import metrics

text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, random_state=42))])

# Learn from the model done by passing our training set to the fit method
text_clf.fit(twenty_train.data, twenty_train.target)  
predicted = text_clf.predict(twenty_test.data)

print('The performance of the SGDClassifier on the twenty_test data is = ', np.mean(predicted == twenty_test.target))

#print("predicted = ",predicted)

#Text summary report
print('\nMetrics classification report: \n',
      metrics.classification_report(twenty_test.target, predicted, target_names=twenty_test.target_names))


The performance of the SGDClassifier on the twenty_test data is =  0.968592964824

Metrics classification report: 
                     precision    recall  f1-score   support

rec.sport.baseball       0.97      0.96      0.97       397
  rec.sport.hockey       0.97      0.97      0.97       399

       avg / total       0.97      0.97      0.97       796



#### 4. Which category is Hockey? 0 or 1? Which category is baseball?

In [25]:
count_vect = CountVectorizer()
train_count = count_vect.fit_transform(twenty_train.data)

train_count.shape 

# There are 1197 arcticles, 18571 words or stems

#twenty_test.target_names
#print(twenty_train.target_names[twenty_train.target[1]])

# The target attribute is the integer index of the category

print("First ten target index list",  twenty_train.target[:10],'\n')
print("The category name for ",twenty_train.target[0], " is: ", twenty_train.target_names[twenty_train.target[0]])
print("The category name for ",twenty_train.target[1], " is: ", twenty_train.target_names[twenty_train.target[1]])


First ten target index list [0 1 0 1 0 1 0 1 1 1] 

The category name for  0  is:  rec.sport.baseball
The category name for  1  is:  rec.sport.hockey


In [26]:
# Which category is Hockey? 0 or 1? Which category is baseball?

predict_h = text_clf.predict(["Hockey"])

print("Hockey belongs to :", predict_h[0]," (", twenty_train.target_names[predict_h[0]],")" )

Hockey belongs to : 1  ( rec.sport.hockey )


In [27]:
#Which category is baseball?
predict_b  = text_clf.predict(["Baseball"])
print("Baseball belongs to :", predict_b[0]," (", twenty_train.target_names[predict_b[0]],")" )

Baseball belongs to : 0  ( rec.sport.baseball )


In [28]:
from pprint import pprint

pprint(list(twenty_train.target_names))


['rec.sport.baseball', 'rec.sport.hockey']
